In [ ]:
!pip3 install "dask[dataframe]" gensim

In [178]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.utils import tokenize
import dask.dataframe as dd
import tqdm
from gensim.models.phrases import Phrases
# from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM, AutoModelForNextSentencePrediction
import torch
from spellchecker import SpellChecker
import fasttext
from nltk.stem import WordNetLemmatizer


# from datasets import load_dataset, load_metric

In [2]:
history = dd.read_csv('~/Downloads/all_requests.csv')
history.head()

,Unnamed: 0,wbuser_id,UQ,cnt,locale,weekday,time
0,0,37bc0ce12ffabce1b1882e66d461ed0e,тапочки женские домашние,1933,Ru,0,10:48:53
1,1,4636a6706e6736d818816d8657565aa2,чехол для бейджика,1513,Ru,0,10:48:53
2,2,708f4040baf99acfc9496563edff1b1a,GUESS,4,Ru,0,10:48:53
3,8,4d269edf958e9b338258d5303a92c124,чехол на айфон 11,64677,Ru,0,10:48:53
4,9,e2382a1810411d2860da0c2e75c5b695,ботинки женские мембрана,507,Ru,0,10:48:53


In [3]:
history[history.wbuser_id == '4636a6706e6736d818816d8657565aa2'].head(100, 10)

/usr/local/lib/python3.9/site-packages/dask/dataframe/core.py:6926: UserWarning: Insufficient elements for `head`. 100 elements requested, only 4 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,Unnamed: 0,wbuser_id,UQ,cnt,locale,weekday,time
1,1,4636a6706e6736d818816d8657565aa2,чехол для бейджика,1513,Ru,0,10:48:53
27458,46076,4636a6706e6736d818816d8657565aa2,бейджик для пропуска,1483,Ru,0,10:49:20
68087,113790,4636a6706e6736d818816d8657565aa2,чехол для пропуска прозрачный,175,Ru,0,10:49:53
72169,120591,4636a6706e6736d818816d8657565aa2,чехол для пропуска в школу,396,Ru,0,10:49:59


In [169]:
lengths = []
with tqdm.tqdm(total=dataset_length) as pbar:
    for text in history['UQ']:
        pbar.update(1)
        lengths.append(len(list(tokenize(str(text).lower()))))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29665474/29665474 [02:49<00:00, 175175.79it/s]


In [322]:
np.median(np.array(lengths)), np.max(np.array(lengths))

(3.0, 51)

In [4]:
dataset_length = len(history.index)
dataset_length

29665474

In [341]:
class HistorySentences(object):
    def __init__(self):
        pass

    def __iter__(self):
        # todo: lock or queue ?
        with tqdm.tqdm(total=dataset_length) as pbar:
            for text in history['UQ']:
                pbar.update(1)
                yield list(tokenize(str(text).lower()))
                

model = Word2Vec(sentences=HistorySentences(), window=4, min_count=1, workers=1, epochs=10)

100%|████████████████████████████████████| 29665474/29665474 [04:38<00:00, 106602.04it/s]


In [342]:
model.save("word2vec_clean10.model")

In [350]:
model.wv.most_similar('самсунг', topn=15)

[('samsung', 0.9157117009162903),
 ('самсун', 0.8189823031425476),
 ('гелакси', 0.8034881353378296),
 ('галакси', 0.7869516015052795),
 ('хонор', 0.7755129337310791),
 ('редми', 0.7578464150428772),
 ('оппо', 0.7550062537193298),
 ('айфон', 0.7400476932525635),
 ('реалми', 0.7317530512809753),
 ('нокиа', 0.7306009531021118),
 ('самснг', 0.7184420228004456),
 ('nokia', 0.7173134684562683),
 ('honor', 0.7064416408538818),
 ('мейзу', 0.7049044370651245),
 ('самунг', 0.7044389843940735)]

In [351]:
model.predict_output_word(['samsung'])

[('galaxy', 0.9924889),
 ('j', 0.001648641),
 ('a', 0.0016327921),
 ('а', 0.00044941602),
 ('tab', 0.00016070121),
 ('напланшет', 5.938427e-05),
 ('sxc', 5.646377e-05),
 ('галаки', 4.6592453e-05),
 ('фе', 4.171397e-05),
 ('пулт', 3.584834e-05)]

In [360]:
model.wv.most_similar('iphone', topn=15)

[('айфон', 0.928081750869751),
 ('афон', 0.7745661735534668),
 ('ipad', 0.7561880946159363),
 ('airpods', 0.7535548806190491),
 ('iphon', 0.7273507714271545),
 ('nokia', 0.7153849601745605),
 ('хонор', 0.709322452545166),
 ('redmi', 0.7023482918739319),
 ('honor', 0.6831232905387878),
 ('айпад', 0.6803625822067261),
 ('samsung', 0.6779000759124756),
 ('самсунг', 0.6770452260971069),
 ('айфона', 0.6700239777565002),
 ('нокиа', 0.6692715883255005),
 ('аирподс', 0.6684114933013916)]

In [357]:
model.predict_output_word(['айфон'])

[('se', 0.6178241),
 ('xr', 0.21223977),
 ('плюс', 0.049806856),
 ('чехол', 0.030586915),
 ('про', 0.02705141),
 ('се', 0.0020816494),
 ('xs', 0.00187226),
 ('чехо', 0.0013647598),
 ('sпластиковый', 0.0010919037),
 ('s', 0.00094762206)]

In [333]:
from spellchecker import SpellChecker

russian = SpellChecker(language='ru')

misspelled = russian.unknown(['телефон', 'телефое', 'телефрн', 'телефо', 'телифон'])
for word in misspelled:
    # Get the one `most likely` answer
    print(word, russian.correction(word))

    # Get a list of `likely` options
    print(russian.candidates(word))

телефое телефон
{'телефон', 'телефоне'}
телифон телефон
{'телефон'}
телефрн телефон
{'телефон'}
телефо телефон
{'телефон'}


In [349]:
russian.correction('timberland')

'timberland'

In [343]:
nlp = spacy.load("ru_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token, token.pos_)

Apple рассматривает возможность покупки стартапа из Соединённого Королевства за $1 млрд
Apple PROPN
рассматривает VERB
возможность NOUN
покупки NOUN
стартапа NOUN
из ADP
Соединённого ADJ
Королевства PROPN
за ADP
$ SYM
1 NUM
млрд NOUN


# Дальше русское поле экспериментов, земля пухом

In [42]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))
['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

# Download the "glove-twitter-25" embeddings
glove_vectors = gensim.downloader.load('word2vec-ruscorpora-300')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[========------------------------------------------] 17.3% 34.4/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================----------------------------------] 32.7% 64.9/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=======================---------------------------] 47.8% 95.0/198.8MB downloaded

...

[================================------------------] 65.2% 129.7/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[============================================------] 89.9% 178.7/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 198.8/198.8MB downloaded


In [55]:
glove_vectors.most_similar('ботинок_NOUN')

[('сапог_NOUN', 0.8237730264663696),
 ('туфля_NOUN', 0.7942396998405457),
 ('башмак_NOUN', 0.788751482963562),
 ('носок_NOUN', 0.7704805135726929),
 ('брюки_NOUN', 0.7260115146636963),
 ('кед_NOUN', 0.7176979780197144),
 ('полуботинок_NOUN', 0.7108590602874756),
 ('кроссовок_NOUN', 0.7046354413032532),
 ('валенок_NOUN', 0.6945410966873169),
 ('обувь_NOUN', 0.6933269500732422)]

In [54]:
glove_vectors.key_to_index

{'весь_DET': 0,
 'человек_NOUN': 1,
 'мочь_VERB': 2,
 'год_NOUN': 3,
 'сказать_VERB': 4,
 'время_NOUN': 5,
 'говорить_VERB': 6,
 'становиться_VERB': 7,
 'знать_VERB': 8,
 'самый_DET': 9,
 'дело_NOUN': 10,
 'день_NOUN': 11,
 'жизнь_NOUN': 12,
 'рука_NOUN': 13,
 'очень_ADV': 14,
 'первый_ADJ': 15,
 'давать_VERB': 16,
 'новый_ADJ': 17,
 'слово_NOUN': 18,
 'иметь_VERB': 19,
 'большой_ADJ': 20,
 'идти_VERB': 21,
 'глаз_NOUN': 22,
 'место_NOUN': 23,
 'лицо_NOUN': 24,
 'видеть_VERB': 25,
 'хотеть_VERB': 26,
 'понимать_VERB': 27,
 'должный_ADJ': 28,
 'работа_NOUN': 29,
 'каждый_DET': 30,
 'друг_NOUN': 31,
 'голова_NOUN': 32,
 'дом_NOUN': 33,
 'оставаться_VERB': 34,
 'сторона_NOUN': 35,
 'начинать_VERB': 36,
 'думать_VERB': 37,
 'хорошо_ADV': 38,
 'жить_VERB': 39,
 'стоять_VERB': 40,
 'спрашивать_VERB': 41,
 'сделать_VERB': 42,
 'выходить_VERB': 43,
 'последний_ADJ': 44,
 'русский_ADJ': 45,
 'сила_NOUN': 46,
 'получать_VERB': 47,
 'какой-то_DET': 48,
 'хороший_ADJ': 49,
 'случай_NOUN': 50,
 'во

In [74]:
ft_model = gensim.models.fasttext.FastTextKeyedVectors.load('/Users/a19531364/Downloads/213/model.model')

In [166]:
w2v_ft = gensim.models.Word2Vec()
w2v_ft.wv = glove_vectors

In [167]:
w2v_ft.predict_output_word(['крем'], topn=50)

RuntimeError: Parameters required for predicting the output words not found.

In [149]:
ft_model.most_similar('прыщи', topn=30)

[('прыщик', 0.829816460609436),
 ('прыщ', 0.7604216933250427),
 ('прыще', 0.6973095536231995),
 ('высыпание', 0.6415020823478699),
 ('сыпь', 0.6245195865631104),
 ('пятнышко', 0.6045306921005249),
 ('язвочка', 0.5864608883857727),
 ('волдырь', 0.5853763222694397),
 ('веснушка', 0.5824287533760071),
 ('шелушиться', 0.5791725516319275),
 ('шелушаться', 0.5743127465248108),
 ('шелушение', 0.571351170539856),
 ('покраснение', 0.5696857571601868),
 ('акне', 0.567973792552948),
 ('гнойничок', 0.5648896098136902),
 ('зудеть', 0.5603037476539612),
 ('язвочки', 0.5575239062309265),
 ('родинка', 0.5522987842559814),
 ('покраснений', 0.5491927862167358),
 ('угорь', 0.5466403961181641),
 ('бородавка', 0.5422810912132263),
 ('морщинка', 0.5414475798606873),
 ('шелушенить', 0.539821982383728),
 ('зуд', 0.5397871136665344),
 ('прыть', 0.5364706516265869),
 ('синяк', 0.5352572202682495),
 ('шрам', 0.5257017016410828),
 ('угревой', 0.5205960869789124),
 ('опрелость', 0.5173868536949158),
 ('папиллом', 

In [288]:
model.predict_output_word(['брюки', 'мужские'], topn=50)

[('классические', 0.007283046),
 ('спортивные', 0.0037131612),
 ('вельветовые', 0.0023478859),
 ('карго', 0.0014761203),
 ('утепленные', 0.001331359),
 ('пижамные', 0.0008058917),
 ('зауженные', 0.00069722335),
 ('камуфляжные', 0.00040778384),
 ('клетчатые', 0.00038952043),
 ('широкие', 0.00036948553),
 ('льняные', 0.00036472222),
 ('джоггеры', 0.0003577866),
 ('софтшелл', 0.00034035332),
 ('утепленые', 0.00031298024),
 ('бананы', 0.00029896045),
 ('летние', 0.0002979574),
 ('костюмные', 0.00028856113),
 ('утеплённые', 0.00026405748),
 ('тактические', 0.0002626477),
 ('чиносы', 0.00024826988),
 ('клешеные', 0.00023934698),
 ('трикотажные', 0.00023620999),
 ('карманами', 0.00020747847),
 ('рабочие', 0.00019680308),
 ('манжете', 0.00018896181),
 ('класические', 0.00018304723),
 ('укороченные', 0.0001745855),
 ('нейлоновые', 0.00017290332),
 ('галифе', 0.00017107182),
 ('reebok', 0.0001678061),
 ('утеплением', 0.0001639607),
 ('прямые', 0.0001594157),
 ('поварские', 0.00015770676),
 ('бол

In [127]:
model.predict_output_word?

In [112]:
query = glove_vectors['телефон_NOUN']
glove_vectors.most_similar(query)

[('телефон_NOUN', 1.0000001192092896),
 ('телефонный_ADJ', 0.7343827486038208),
 ('звонить_VERB', 0.6245343089103699),
 ('мобильник_NOUN', 0.6179441809654236),
 ('позвонить_VERB', 0.6179267168045044),
 ('мобильный_ADJ', 0.6130176186561584),
 ('дозваниваться_VERB', 0.6114135980606079),
 ('звонок_NOUN', 0.5979376435279846),
 ('сотовый_ADJ', 0.591869056224823),
 ('ньюс::рум_NOUN', 0.5876462459564209)]

In [231]:
!pip3 install stanfordnlp spacy_stanfordnlp

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 158 kB 1.7 MB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using 

In [278]:
nlp = spacy.load("ru_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.lemma_, token.similarity(nlp('iphone')))

Apple рассматривает возможность покупки стартапа из Соединённого Королевства за $1 млрд
apple 0.749515060353256
рассматривать 0.09261202608900057
возможность -0.03689531183280221
покупка -0.00018228096888834176
стартап -0.034750160464460736
из -0.010565416953560667
соединённого -0.09452835581731267
королевство -0.07184442186420445
за 0.04444165293583694
$ 0.20292149184124966
1 0.05027686038687083
млрд 0.23706291106173324


/var/folders/04/j48spdmx7_7_h5jkgynrt6780000gq/T/ipykernel_27590/1229185882.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(token.lemma_, token.similarity(nlp('iphone')))


In [262]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x2475e0f40>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x2475e0fa0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1ddd85430>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ddd8d100>),
 ('lemmatizer', <spacy.lang.ru.lemmatizer.RussianLemmatizer at 0x1dd5245c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1de11f4a0>)]

In [263]:
nlp('iphone').similarity(nlp('Мобильные телефоны')), nlp('iphone').similarity(nlp('телефон'))

/var/folders/04/j48spdmx7_7_h5jkgynrt6780000gq/T/ipykernel_27590/2160491651.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp('iphone').similarity(nlp('Мобильные телефоны')), nlp('iphone').similarity(nlp('телефон'))


(0.4437573351878108, 0.6587200652436649)

In [286]:
[token.lemma_ for token in nlp('телефончики в кармане')]

['телефончик', 'в', 'карман']

In [344]:
!python3 -m spacy download ru_core_news_lg

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 514.5 MB 31 kB/s               
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
